In [18]:
from usda_mmn_utils import *
import openpyxl

In [19]:
markets_df = get_markets_list()
print(markets_df.head())
print(markets_df.tail())


  slug_id                  market_name
0    2277               Atlanta fruits
1    2278           Atlanta vegetables
2    2279  Atlanta onions and potatoes
3    2280                 Atlanta nuts
4    2281             Baltimore fruits
    slug_id                                        market_name
137    3706    Hamburg, Germany Terminal Nut Prices (HA_FV201)
138    3716                                Tokyo, Japan fruits
139    3717                            Tokyo, Japan vegetables
140    3718                   Tokyo, Japan onions and potatoes
141    3719  Tokyo, Japan Ornamental Terminal Prices (TK_FV...


In [20]:
commodities_list = get_commodities_list(slug_id = markets_df.iloc[0]['slug_id'])
print(commodities_list)


['Limes', 'Lemons', 'Grapes', 'Plantains', 'Coconuts', 'Apples', 'Papaya', 'Bananas', 'Peaches', 'Kiwifruit', 'Mangoes', 'Oranges', 'Guava', 'Grapefruit', 'Avocados', 'Cantaloupes', 'Blueberries', 'Raspberries', 'Pears', 'Strawberries', 'Blackberries', 'Honeydews', 'Pineapples', 'Dates', 'Watermelons', 'Carambola - Star Fruit', 'Rambutan']


In [21]:
prices_df = get_prices(commodity=commodities_list[0], slug_id=markets_df.iloc[0]['slug_id'], start_year=2020, end_year=2022)

In [22]:
print(prices_df.head())

  report_date report_begin_date report_end_date       published_date  \
0  12/31/2020        12/31/2020      12/31/2020  12/31/2020 00:00:00   
1  12/31/2020        12/31/2020      12/31/2020  12/31/2020 00:00:00   
2  12/31/2020        12/31/2020      12/31/2020  12/31/2020 00:00:00   
3  12/31/2020        12/31/2020      12/31/2020  12/31/2020 00:00:00   
4  12/31/2020        12/31/2020      12/31/2020  12/31/2020 00:00:00   

                           office_name office_state  office_city market_type  \
0  Atlanta (Forest Park), Georgia - SC           GA  Forest Park    Terminal   
1  Atlanta (Forest Park), Georgia - SC           GA  Forest Park    Terminal   
2  Atlanta (Forest Park), Georgia - SC           GA  Forest Park    Terminal   
3  Atlanta (Forest Park), Georgia - SC           GA  Forest Park    Terminal   
4  Atlanta (Forest Park), Georgia - SC           GA  Forest Park    Terminal   

      market_location_name market_location_state  ... high_price  \
0  Atlanta Termina

In [23]:
veg_market = "2278"
commodities_list = get_commodities_list(slug_id = veg_market)
print(commodities_list)

['Cabbage', 'Okra', 'Beans, Round Green Type', 'Marjoram', 'Asparagus', 'Spinach', 'Oregano', 'Tomatoes', 'Lemon Grass', 'Tomatillos', 'Squash, Yellow Straightneck', 'Bean Sprouts', 'Rutabagas', 'Parsley', 'Lettuce, Romaine', 'Cucumbers', 'Celery', 'Artichokes', 'Malanga/ Yautia', 'Greens, Swiss Chard', 'Taro', 'Chives', 'Peppers, Finger Hot', 'Chinese Cabbage', 'Daikon', 'Peppers, Bell Type', 'Mushrooms', 'Parsnips', 'Rosemary', 'Peppers, Serrano', 'Beets', 'Chayote', 'Dill', 'Beans, Cranberry Type', 'Cauliflower', 'Beans, Flat Green Type', 'Yuca (Cassava)', 'Watercress', 'Escarole', 'Celeriac (Celery Root)', 'Sweet Potatoes', 'Lettuce, Green Leaf', 'Radicchio', 'Carrots', 'Lettuce, Red Leaf', 'Tomatoes, Plum Type', 'Cilantro', 'Peppers, Habanero', 'Broccoli', 'Squash, Zucchini', 'Dry Eschallot', 'Mint', 'Eggplant', 'Arugula', 'Bok Choy', 'Tomatoes, Grape Type', 'Peas, Snow', 'Squash, Acorn', 'Endive', 'Alfalfa Sprouts', 'Turnips', 'Jicama', 'Squash, Yellow Crookneck', 'Sage', 'Basil'

In [24]:
commodity = "Tomatoes"
prices_df = get_prices_1y(commodity=commodity, slug_id=veg_market, year=2022)
print(prices_df.keys())

Index(['report_date', 'report_begin_date', 'report_end_date', 'published_date',
       'office_name', 'office_state', 'office_city', 'market_type',
       'market_location_name', 'market_location_state', 'market_location_city',
       'slug_id', 'slug_name', 'report_title', 'group', 'category',
       'commodity', 'variety', 'repack', 'package', 'storage',
       'transportation_mode', 'grade', 'unit_sales', 'item_size', 'appearance',
       'quality', 'condition', 'organic', 'crop', 'origin', 'district',
       'environment', 'properties', 'low_price', 'high_price',
       'mostly_low_price', 'mostly_high_price', 'market_tone_comments',
       'offerings_comments', 'special_notes', 'reporter_comment',
       'commodity_comments', 'report_narrative', 'report_footnotes'],
      dtype='object')


In [28]:

commodity_col=[]
variety_col=[]
package_col=[]
pounds_col=[]
print("Unique varieties and packaging types")
for variety in prices_df["variety"].unique():
    print("Variety: "+variety)
    for package in prices_df[prices_df["variety"]==variety]["package"].unique():
        print("\t"+package)
        pounds = None
        words_list = (package.replace("/"," ")).split(" ")
        if "lb" in package:
            pounds = int(words_list[words_list.index("lb")-1])
        elif "kg" in package:
            pounds = int(words_list[words_list.index("kg")-1]*2.2)  # 1kg = 2.2lb
        else:
            package_to_pounds = pd.read_csv("packagetype_to_pounds.csv")
            tonw_commodity = package_to_pounds[(package_to_pounds["MARS commodity"] == commodity)
                                               & (package_to_pounds["MARS variety"] == variety)].loc[0]["table of net weights commodity"]
            if tonw_commodity is not None:  # TONW = Table of Net Weights
                tonw_df = pd.read_excel("table of net weights corrected names.xlsx")
                sub_tonw_df = tonw_df[(tonw_df["Commodity"] == tonw_commodity)
                                      & (tonw_df["Pack Description"] == package)]
                if not sub_tonw_df.empty:
                    pound = int(sub_tonw_df.iloc[0]["Package Weight"])

        commodity_col.append(commodity)
        variety_col.append(variety)
        package_col.append(package)
        pounds_col.append(pounds)

pound_mapper = pd.DataFrame({"commodity":commodity_col, "variety":variety_col, "package":package_col, "pounds":pounds_col})
print(pound_mapper)

Unique varieties and packaging types
Variety: VINE RIPES
	25 lb cartons loose
	cartons 2 layer
	5 kg/11 lb flats
	20 lb cartons loose
Variety: MATURE GREENS
	25 lb cartons loose
Variety: IMMATURE GREENS
	25 lb cartons loose
  commodity          variety              package  pounds
0  Tomatoes       VINE RIPES  25 lb cartons loose    25.0
1  Tomatoes       VINE RIPES      cartons 2 layer     NaN
2  Tomatoes       VINE RIPES     5 kg/11 lb flats    11.0
3  Tomatoes       VINE RIPES  20 lb cartons loose    20.0
4  Tomatoes    MATURE GREENS  25 lb cartons loose    25.0
5  Tomatoes  IMMATURE GREENS  25 lb cartons loose    25.0


In [ ]:
pound_mapper = pd.DataFrame(columns=)